In [1]:
from trainer import Trainer
from zytlib import vector, path
from zytlib import Logger
from zytlib.table import table
from zytlib.visual.animation import TimeStamp, ScatterAnimation
from tqdm import tqdm
import argparse
import torch
from torchfunction.device import todevice, get_device
from matplotlib import pyplot as plt
import seaborn as sns
import functools
import numpy as np
from matplotlib.animation import FuncAnimation
from matplotlib import animation
from utils import SOR
from torchfunction.inspect import get_shape
from collections import OrderedDict
import math
%config InlineBackend.figure_format = 'svg'
device = get_device("cuda")

Using GPU device 0...


In [19]:
dataset_path = "dataset/dataset_only_rank3_with_rank_10_delay_15_last_delay_50.db"

In [20]:
model = Trainer.load("model/2021-1111-14.pt", datapath=dataset_path)
model.eval()

datapath : dataset/dataset_only_rank3_with_rank_10_delay_15_last_delay_50.db
delta_t : 20
tau : 100
embedding_size : 1024
decoder_dim : 512
noise_sigma : 0.05
input_dim : 6
batch_size : 32
linear_decoder : True
device : cuda
embedding : tensor([[ 0.0186, -0.0156, -0.0099,  ...,  0.0778, -0.0111, -0.0148],
        [ 0.0201, -0.0297,  0.0112,  ...,  0.0508, -0.0064,  0.0204],
        [ 0.0015, -0.0141,  0.0211,  ..., -0.0270,  0.0047,  0.0352],
        [-0.0186,  0.0156,  0.0099,  ..., -0.0778,  0.0111,  0.0148],
        [-0.0201,  0.0297, -0.0112,  ..., -0.0508,  0.0064, -0.0204],
        [-0.0015,  0.0141, -0.0211,  ...,  0.0270, -0.0047, -0.0352]],
       device='cuda:0')
learning_rate : 2e-05
is_embedding_fixed : True
l2_reg : 1e-05
encoder_bias : False
decoder_bias : False
encoder_to_decoder_equal_space : False
encoder_max_rank : 5
decoder_max_rank : -1
freeze_parameter : ['decoder']


In [21]:
result = vector()

hidden_state_all = vector()
ground_truth_all = vector()

with torch.no_grad():

    for batch in model.test_dataloader:
        input_encoder, length, ground_truth_tensor, ground_truth_length, last_delay_length = todevice(batch, device=model.hyper["device"])
        hidden_state, final_state = model.encoder(input_encoder, length)
        hidden_state_all.append(hidden_state)
        ground_truth_all.append(ground_truth_tensor)
        decoded_seq, _ = model.decoder(final_state, torch.zeros_like(ground_truth_tensor).fill_(-1), ground_truth_length, teaching_forcing_ratio=0.0)
        decoded_item = decoded_seq.argmax(-1)
        for i in range(hidden_state.shape[0]):
            result.append(vector(ground_truth_tensor[i, :], decoded_item[i, :], ground_truth_tensor[i, :]==decoded_item[i, :]).map(lambda x: x.cpu().numpy()))

In [22]:
result[:, 2].map(vector).mean(dim=0)

[1.0, 1.0, 0.9351851851851852]